In [369]:
import joblib
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.sparse import csr_matrix

[nltk_data] Downloading package punkt to C:\Users\zxc
[nltk_data]     ghoul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import faiss

In [3]:
import torch
from transformers import BertTokenizer, BertModel

In [371]:
db_name = 'recipe.db'
conn = sqlite3.connect(db_name)

table_name = 'recipe'

#забираем доступные категории
ingredients_sql = f"select i.id, i.name_ingr from ingredient i"

loaded_categories = pd.read_sql(ingredients_sql, conn)

ids_ingr = []
for i in range(len(loaded_categories)):
    ids_ingr.append(str(loaded_categories.id[i]))

#print(ids_ingr)

#забираем способ приготовления рецепта с категориями из доступных с id категорий
sql = f"SELECT r.id, r.manual, (SELECT group_concat(ri.ingr_id , ', ') " \
      f"from recipe_ingredients ri " \
      f"WHERE ri.recipe_id = r.id AND " \
      f"ri.ingr_id in ({', '.join(ids_ingr)})) " \
      f"AS ingredients from recipe r"

loaded_data = pd.read_sql(sql, conn)

#pandas data frame
conn.close()

In [375]:
loaded_data

,id,manual,ingredients,1,2,3,4,5,6,7,...,2770,2771,2772,2773,2774,2775,2776,2777,2778,2779
0,1,Шоколад разломать на кусочки и вместе со сливо...,"1, 2, 3, 4, 5, 6",1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Положите весь творог в кастрюльку и разомните ...,"4, 5, 7, 8, 9",0,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,Вскипятите воду в большой кастрюле и сварите п...,"10, 11, 12, 13, 14, 15, 16, 17, 2, 4",0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Разогреть духовку. Отделить белки от желтков. ...,"18, 19, 20, 4, 5, 8",0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Взбить яйца с сахаром.\n\nПостепенно ввести му...,"16, 19, 21, 4, 5, 8",0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6990,6996,"Печенку посолить, поперчить, посыпать специями...","102, 19, 195, 2, 32, 412, 59, 73",0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6991,6997,Крабовые палочки мелко нарезать или порубить в...,"16, 17, 183, 26, 294, 37, 4",0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6992,6998,"Чечевицу промыть, залить водой на 4 см выше ур...","11, 12, 159, 16, 161, 17, 1875, 19, 2188, 34, ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6993,6999,Нарезать филе средними полосками. Морковь наре...,"11, 16, 19, 32, 45, 47, 658, 73, 87",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [373]:
for i in range(len(ids_ingr)):
    val = []
    for j in range(len(loaded_data)):
        spl = str(loaded_data.ingredients[j]).split(', ')
        val.append(int(str(ids_ingr[i]) in spl))
    loaded_data.insert(loc=len(loaded_data.columns) , column=ids_ingr[i], value=val)

C:\Users\zxc ghoul\AppData\Local\Temp\ipykernel_10152\3739734041.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  loaded_data.insert(loc=len(loaded_data.columns) , column=ids_ingr[i], value=val)
C:\Users\zxc ghoul\AppData\Local\Temp\ipykernel_10152\3739734041.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  loaded_data.insert(loc=len(loaded_data.columns) , column=ids_ingr[i], value=val)
C:\Users\zxc ghoul\AppData\Local\Temp\ipykernel_10152\3739734041.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usuall

In [374]:
#делим dataset на тренировочный и тестовый
X_train,X_test,y_train,y_test = train_test_split(loaded_data["manual"], loaded_data[ids_ingr],test_size=0.3,random_state=42)

In [376]:
class BertVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny')
        self.model = BertModel.from_pretrained('cointegrated/rubert-tiny')
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        vectors = []
        for text in X:
            text_vector = self.__vectorize__(text, model=self.model, tokenizer=self.tokenizer)
            vectors.append(text_vector)

        # Преобразование списка в разреженную матрицу
        X_sparse = csr_matrix(vectors)

        return X_sparse

    def __vectorize__(self, text, model, tokenizer):
        t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**{k: v.to(model.device) for k, v in t.items()})
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings[0].cpu().numpy()

In [377]:
class FaissKNeighbors:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k
        self.vectorizer = BertVectorizer()
        self.normalizer = Normalizer()
        self.knn_neighbors = None  # Добавленный атрибут
        self.text_mapping = {}  # Маппинг между векторами и текстовыми описаниями

    def fit(self, X, y):
        X_vectors = self.vectorizer.transform(X)
        X_dense = X_vectors.toarray().astype(np.float32)

        X_normalized = self.normalizer.fit_transform(X_dense)

        dimension = X_normalized.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(X_normalized)

        self.y = np.asarray(y)  # Преобразование меток в массив numpy

        # Сохранение маппинга между векторами и текстовыми описаниями
        for i, text in enumerate(X):
            self.text_mapping[i] = text

        return self

    def predict(self, recipe):
        recipe_vector = self.vectorizer.transform([recipe])
        recipe_vector = self.normalizer.transform(recipe_vector)
        recipe_dense = recipe_vector.toarray().astype(np.float32)

        distances, indices = self.index.search(recipe_dense, self.k)

        knn_labels = self.y[indices]

        self.knn_neighbors = knn_labels.tolist()  # Сохранение ближайших соседей в виде списка

        #Преобразование векторов обратно в текст
        knn_texts = [self.text_mapping[i] for i in indices[0]]

        return knn_texts

    def get_top_recipes(self, recipe_list, k=5):
        # Получаем предсказанных соседей для каждого рецепта в списке
        neighbors_list = [self.predict(recipe) for recipe in recipe_list]

        # Считаем количество вхождений каждого рецепта в список соседей
        recipe_counts = {}
        for neighbors in neighbors_list:
            for recipe in neighbors:
                if recipe in recipe_counts:
                    recipe_counts[recipe] += 1
                else:
                    recipe_counts[recipe] = 1

        # Сортируем рецепты по количеству вхождений в общий список соседей
        sorted_recipes = sorted(recipe_counts.items(), key=lambda x: x[1], reverse=True)

        # Выбираем топ-K наиболее подходящих рецептов
        top_recipes = [recipe for recipe, _ in sorted_recipes[:self.k]]

        return top_recipes

In [378]:
knn_faiss = FaissKNeighbors()

knn_faiss.fit(X_train, y_train)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: 

In [154]:
X_test[8]

'Яйца взбить с сахаром до белой пены. Добавить 2 стакана просеянной муки, соду, растопленное остывшее сливочное масло, сметану и тщательно перемешать (лучше миксером).\n\nТесто разделить на две равные части. В одну часть добавить 2 столовые ложки муки, в другую 2 столовые ложки какао. Перемешать, чтобы не было комочков. Тесто должно быть консистенции негустой сметаны.\n\nШирокую форму (26–28 см), смазать маслом. Вливать в центр поочередно небольшие порции разного теста (по столовой ложке). Не перемешивать.\n\nВыпекать в предварительно разогретой духовке при температуре 200 градусов в течение получаса. Если верх пирога уже пропечется, а середина еще нет — следует накрыть пирог фольгой, уменьшить температуру до 180 градусов и выпекать до готовности.\n\n'

In [345]:
recipe = X_test[8]
print(recipe)
similar_neighbors = knn_faiss.predict(recipe)

Яйца взбить с сахаром до белой пены. Добавить 2 стакана просеянной муки, соду, растопленное остывшее сливочное масло, сметану и тщательно перемешать (лучше миксером).

Тесто разделить на две равные части. В одну часть добавить 2 столовые ложки муки, в другую 2 столовые ложки какао. Перемешать, чтобы не было комочков. Тесто должно быть консистенции негустой сметаны.

Широкую форму (26–28 см), смазать маслом. Вливать в центр поочередно небольшие порции разного теста (по столовой ложке). Не перемешивать.

Выпекать в предварительно разогретой духовке при температуре 200 градусов в течение получаса. Если верх пирога уже пропечется, а середина еще нет — следует накрыть пирог фольгой, уменьшить температуру до 180 градусов и выпекать до готовности.




In [364]:
recipes = X_test[5:6].tolist()
similar_neighbors = knn_faiss.get_top_recipes(recipes)

In [365]:
recipes

['Свеклу и картофель поперчить, смазать оливковым маслом и завернуть в фольгу.\n\nСолить не нужно — соль замедляет процесс приготовления.\n\nПоложить в духовку, разогретую до 180 С.\n\nКогда овощи можно будет проткнуть деревянной шпажкой (картофель — примерно через 40 минут, свеклу — примерно через 2 часа), овощи вынуть и дать им остыть.\n\nТеперь почистите свеклу, натрите ее на терке или мандолине соломкой и залейте минеральной водой. Двадцати минут хватит, чтобы получился настой.\n\nВ это время одинаковой соломкой нарежьте или натрите картофель, огурцы и редиску.\n\nНарубите зеленый лук и укроп.\n\nНарезать свекольную ботву соломкой удалив сердцевину листа.\n\nСварите яйца, добавив соль в воду — так скорлупа не потрескается. Обдайте яйца холодной водой, почистите и разрежьте на половинки.\n\nТеперь сделайте концентрат — свекольный настой перелейте через дуршлаг в отдельную посуду, оставив свеклу. Это нужно для того, чтобы не нарушить фактуру — представьте, если смешать вместе, то все

In [366]:
similar_neighbors

['В большую кастрюлю налить оливковое масло, добавить к нему по 100 г лука, чеснока, моркови, сельдерея и фенхеля. Овощи нарезать большими кусочками, чтобы они не разваривались. Чеснок не делить на дольки, разрезать чесночину пополам и положить в кастрюлю. Посолить, перемешать, добавить кориандр, гвоздику, анис. Перемешать, убавить огонь и дать постоять пять минут.\n\nВзять двух осьминогов. Осьминоги — самый важный ингредиент. Лучше всего португальские осьминоги. Большие, нежные, не требуют больших затрат. Головы осьминогов отрезать и выбросить. Обычно голову используют, но в этом блюде нужны только длинные щупальца.\n\nПрибавить под кастрюлей огонь, положить осьминогов и залить водой, чтобы вода накрыла овощи и осьминогов. Когда варево закипит, снять кастрюлю с огня, положить на смесь бумагу с дыркой посередине — как крышку. Это спасет осьминога от \xadпере\xadсыхания. Затем поставить кастрюлю в духовку на полтора часа при температуре 135 градусов. В это время сделать заправку и соус 

In [370]:
joblib.dump(knn_faiss, "./knn.joblib")

['./knn.joblib']